# 賃料推定エンジンの構築

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV

In [2]:
data = pd.read_csv('train_aparts.csv')

In [31]:
data2.head(20)

,yachin,size,years,loft,all_floor,target_floor,access_minutes,land_price,dummy_ivc,12SLDK,...,2SLDK,3DK,3LDK,3SLDK,4LDK,ワンルーム,千葉県,埼玉県,東京都,神奈川県
0,5.8,26.49,0.0,0.0,2.0,2.0,14.0,82500.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,4.3,8.80,1.0,1.0,2.0,2.0,8.0,419000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,4.2,13.81,0.0,0.0,3.0,1.0,5.0,326000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,4.4,13.81,0.0,0.0,3.0,2.0,5.0,326000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,4.1,14.09,0.0,0.0,3.0,1.0,5.0,326000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5,4.1,14.01,1.0,0.0,3.0,1.0,5.0,326000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6,4.4,13.81,0.0,0.0,3.0,1.0,5.0,326000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7,4.5,14.25,0.0,0.0,3.0,1.0,5.0,326000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
8,4.5,14.09,0.0,0.0,3.0,2.0,5.0,326000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9,4.6,13.81,0.0,0.0,3.0,2.0,5.0,326000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [7]:
pd.get_dummies(data.loc[:, 'prefecture']).head()

,千葉県,埼玉県,東京都,神奈川県
0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [12]:
pd.get_dummies(data.loc[:, 'type']).head()

,12SLDK,1DK,1K,1LDK,1LK,1SDK,1SK,1SLDK,1SLK,2DK,2K,2LDK,2LK,2SDK,2SLDK,3DK,3LDK,3SLDK,4LDK,ワンルーム
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [29]:
data_to_dummy = pd.concat([data, pd.get_dummies(data.loc[:, 'type']), pd.get_dummies(data.loc[:, 'prefecture'])], axis = 1).drop(['type', 'hougaku', 'kouzo', 'prefecture'], axis = 1)

In [30]:
data2.to_csv('train_aparts_v2.csv')